### Approch
<b>step 1:</b> after loading the data, droping contact and duration as calling on a cell or landline won't effect the buying decision and duration highly effects the outcome as per the website.
<b>step 2:</b> removing rows with unknown values in all three job, education and poutcome columns.
<b>step 3:</b> Preprocessing: Encoding categorical columns
<b>step 4:</b> Dividing the model into train and test data
<b>step 5:</b> Using LogisticRegression to predict the test data
<b>step 6:</b> Improving accuracy by filtering

In [127]:
import pandas as pd
import numpy as np

In [157]:
bank_data=pd.read_csv("https://raw.githubusercontent.com/edyoda/data-science-complete-tutorial/master/Data/bank-full.csv", sep=";")
bank_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [158]:
## Discarding duration and contact. 
bank_data=bank_data.drop(columns=['duration', 'contact'])

In [159]:
#droping below rows with lot of unknow values
lb=bank_data[(bank_data.job=='unknown') & (bank_data.education=='unknown') & (bank_data.poutcome=='unknown')].index
bank_data=bank_data.drop(labels=lb)
bank_data.info()
#from 45211 entries, we are left with 45095 entries

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45095 entries, 0 to 45210
Data columns (total 15 columns):
age          45095 non-null int64
job          45095 non-null object
marital      45095 non-null object
education    45095 non-null object
default      45095 non-null object
balance      45095 non-null int64
housing      45095 non-null object
loan         45095 non-null object
day          45095 non-null int64
month        45095 non-null object
campaign     45095 non-null int64
pdays        45095 non-null int64
previous     45095 non-null int64
poutcome     45095 non-null object
y            45095 non-null object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [160]:
# Encoding categorical columns
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()

In [161]:
object_columns=bank_data.select_dtypes(include=['object']).columns
bank_data[object_columns]=oe.fit_transform(bank_data[object_columns])
bank_data.head()

,age,job,marital,education,default,balance,housing,loan,day,month,campaign,pdays,previous,poutcome,y
0,58,4.0,1.0,2.0,0.0,2143,1.0,0.0,5,8.0,1,-1,0,3.0,0.0
1,44,9.0,2.0,1.0,0.0,29,1.0,0.0,5,8.0,1,-1,0,3.0,0.0
2,33,2.0,1.0,1.0,0.0,2,1.0,1.0,5,8.0,1,-1,0,3.0,0.0
3,47,1.0,1.0,3.0,0.0,1506,1.0,0.0,5,8.0,1,-1,0,3.0,0.0
5,35,4.0,1.0,2.0,0.0,231,1.0,0.0,5,8.0,1,-1,0,3.0,0.0


In [162]:
from sklearn.model_selection import train_test_split

In [163]:
feature_data=bank_data.drop(columns=['y'])
target_data=bank_data.y
trainX, testX, trainY, testY = train_test_split(feature_data, target_data)

In [164]:
from sklearn.linear_model import LogisticRegression

In [165]:
lr=LogisticRegression()

In [166]:
lr.fit(trainX, trainY)
print("Score from logistic regression :", lr.score(testX,testY))

Score from logistic regression : 0.8780379634557389


C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### LogisticRegression is already giving us an accuracy of 87%. We will now use Feature reduction techniques and see if we can modify the results

In [167]:
experiement_bank_data=bank_data.copy()

In [168]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=0.2)

In [169]:
vt_feature_data=vt.fit_transform(experiement_bank_data.drop(columns=['y']))

In [170]:
#out of 14 features, 2 features have been dropped
vt_feature_data.shape

(45095, 12)

In [171]:
vt_trainX, vt_testX, vt_trainY, vt_testY = train_test_split(vt_feature_data, target_data)

In [172]:
lr.fit(vt_trainX,vt_trainY)
print("accuracy after VarianceThreshold: ", lr.score(vt_testX,vt_testY))

accuracy after VarianceThreshold:  0.8805215540180947


C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Accuracy is almost the same. 
### Next we try choosing start small featuring technique

In [180]:
features = list(experiement_bank_data.columns.values)
features.remove('y')

In [181]:
## We are getting max accuracy of 88% using all the features.
x=[]
max_score=0
for feature in features:
    x.append(feature)
    lr = LogisticRegression()
    trainX, testX, trainY, testY = train_test_split(experiement_bank_data[x], experiement_bank_data.y)
    lr.fit(trainX, trainY)
    l_score=lr.score(testX,testY)
    if l_score>max_score:
        max_score=l_score
        maxx=x
        
print("max accuracy of {} is achieved with features as : ".format(max_score),maxx)

C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mo

max accuracy of 0.8863757317722193 is achieved with features as :  ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'day', 'month', 'campaign', 'pdays', 'previous', 'poutcome']


C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [182]:
from mlxtend.feature_selection import SequentialFeatureSelector
sfs = SequentialFeatureSelector(k_features=5, estimator=LogisticRegression())

In [183]:
feature_data=bank_data.drop(columns=['y'])
target_data=bank_data.y
trainX, testX, trainY, testY = train_test_split(feature_data, target_data)
sfs.fit(trainX, trainY)

SequentialFeatureSelector(estimator=LogisticRegression(), k_features=5)

In [186]:
#we will use these features to test test_data
sfs_feature=list(sfs.k_feature_names_)
sfs_feature

['job', 'default', 'month', 'pdays', 'poutcome']

In [187]:
sfs_feature_data=experiement_bank_data[sfs_feature]
sfs_target_data=experiement_bank_data.y
s_trainX, s_testX, s_trainY, s_testY= train_test_split(sfs_feature_data, sfs_target_data)

In [188]:
lr=LogisticRegression()
lr.fit(s_trainX, s_trainY)
lr.score(s_testX, s_testY)

0.8805215540180947

## Hence we are getting a max accuracy of 88% using LogisticRegressions